In [11]:
#IMPORT THIS TO SLICER PYTHON INTERPRETOR TO SET CAMERA PARAMETERS

import slicer
import math
# Get the first active 3D view
view = slicer.app.layoutManager().threeDWidget(0).threeDView()
# Get the camera node
cameraNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLCameraNode")
view.resetFocalPoint()
view.lookFromViewUp(slicer.qMRMLViewNode.Posterior)
if cameraNode:
    # Set camera properties for coronal view
    cameraNode.SetPosition([32.36507129475949, -140.45358451935044, -361.3561768450015])
    cameraNode.SetFocalPoint([1.46, -172.56, -445.58])
    cameraNode.SetViewUp([0.92474159, -0.37009835, -0.08877048])
    cameraNode.SetRoll(51.79640336331715)
    # Refresh the view to see the changes
    view.forceRender()
else:
    print("No camera node found.")

# case2 example
# Camera View Up: [-0.1208574  -0.01827804  0.99250159]
# Camera Position: [ -15.42091046  156.052597   -404.05671836]
# Camera Focal Point: [   5.75707529 -173.82832368 -407.55299543]
# Camera View Up: [-0.1208574  -0.01827804  0.99250159]



ModuleNotFoundError: No module named 'slicer'

In [ ]:
#IMPORT THIS TO SLICER PYTHON INTERPRETOR TO GET CURRENT CAMERA PARAMETERS

import slicer
# Get the first active 3D view
view = slicer.app.layoutManager().threeDWidget(0).threeDView()
# Get the renderer associated with the view
renderer = view.renderWindow().GetRenderers().GetFirstRenderer()
# Get the camera associated with the renderer
activeCameraNode = renderer.GetActiveCamera()
if activeCameraNode:
    # Access the camera properties
    position = activeCameraNode.GetPosition()
    focalPoint = activeCameraNode.GetFocalPoint()
    viewUp = activeCameraNode.GetViewUp()
    CameraRoll = activeCameraNode.GetRoll()
    # Print the camera properties
    print("Camera Position:", position)
    print("Camera Focal Point:", focalPoint)
    print("Camera View Up Vector:", viewUp)
    print("Camera Roll", CameraRoll)
else:
    print("No active camera node found.")


In [ ]:
# Very Simple Example of showing a .VTK file as 3d model with color and opacity options

from mayavi import mlab
vtk_file = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-2-MESAL-5010730-E3/Markups/88-Adam/Aseed(1%).vtk"
src = mlab.pipeline.open(vtk_file)

# Create a surface visualization of the data
surface = mlab.pipeline.surface(src, color=(0, 0, 1), opacity = 0.5)

mlab.show()

In [4]:
### WORKS TO SHOW .VTK FILE AND CURRENT CAMERA PARAMETERS by MOUSE MOVEMENT
from mayavi import mlab

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

    
vtk_file = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-2-MESAL-5010730-E3/Markups/88-Adam/Aseed(1%).vtk"
# Load the VTK file using Mayavi's built-in reader
src = mlab.pipeline.open(vtk_file)
# Create a surface visualization of the data
surface = mlab.pipeline.surface(src, color=(0, 0, 1), opacity = 0.5)

# Attach the callback to the interaction event
mlab.view(azimuth=45, elevation=30, distance=400)
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
# Show the visualization
mlab.show()




Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256 -416.86001587]
Camera View Up: [0. 0. 1.]
Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256 -416.86001587]
Camera View Up: [0. 0. 1.]
Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256 -416.86001587]
Camera View Up: [0. 0. 1.]
Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256 -416.86001587]
Camera View Up: [0. 0. 1.]
Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256 -416.86001587]
Camera View Up: [0. 0. 1.]
Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256 -416.86001587]
Camera View Up: [0. 0. 1.]
Camera Position: [145.9738541  -33.61564633 -70.44985436]
Camera Focal Point: [   4.55249786 -175.03700256

In [1]:
### THIS WORKS TO SHOW .nrrd file
## Keep in mind that the camera information shown isn't identical to slicer

import numpy as np
import SimpleITK as sitk
from mayavi import mlab
from mayavi.mlab import contour3d
def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)
print("Imports and Function are good")

# Load the .nrrd file using SimpleITK
reader = sitk.ImageFileReader()
reader.SetFileName('z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-2-MESAL-5010730-E3/Markups/88-Adam/S-Final-1_1.seg.nrrd')
image = reader.Execute()
data = sitk.GetArrayFromImage(image)
transposed_data = np.transpose(data)
print("Image Transposed")

# Create a figure
mlab.figure(bgcolor=(0, 0, 0))
# Create an isosurface visualization of the segmentation
contour3d(transposed_data, colormap="Blues", opacity = 0.5)
mlab.view(azimuth=45, elevation=30, distance=400)
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
# Customize the visualization settings (colormap, opacity, etc.) as needed
print("Woohoo")
# Show the visualization
mlab.show()


Imports and Function are good
Image Transposed
Woohoo
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal Point: [375.5 263.5 306.5]
Camera View Up: [0. 0. 1.]
Camera Position: [516.92135624 404.92135624 652.91016151]
Camera Focal 

In [14]:
## IN PROGRESS
## OBJECTIVE: Create calulcations for camera parameters

import SimpleITK as sitk
import numpy as np

#Find Center of model

# model_path= "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-32-0.75__QR40RESEARCH__A_80KV_Qr40d_51292112/Markups/88-Jensen/S-Final-2_1.seg.nrrd"
model_path= "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-34-0.5__QR40__RESEARCH__A_80KV_Qr40d_3_52999588/Markups/88-Adam/S-Final-2.nrrd"
model = sitk.ReadImage(model_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)

# focalPoint = activeCameraNode.GetFocalPoint()
# viewUp = activeCameraNode.GetViewUp()
# CameraRoll = activeCameraNode.GetRoll()


distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)
print("Direction: ", direction)
# Center answers: might need to adjust for negatives 
# Origin:  (-167.3, -326.79999999999995, -733.6999999999999)
# Dimensions:  (910, 609, 711)
# Center:  [   5.6  -211.09 -598.61]

Origin:  (-119.00000000000001, -262.9999999999999, -544.0000000000001)
Dimensions:  (634, 476, 518)
Center:  [   1.46 -172.56 -445.58]
Direction:  (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [2]:
## EXAMPLE CODE TO MAKE 3d MODEL of shape 
import numpy as np
from mayavi import mlab

# Create a simple 3D array for demonstration
x, y, z = np.ogrid[-5:5:100j, -5:5:100j, -5:5:100j]
data = np.sin(x * y * z) / (x * y * z)

# Create a Mayavi figure
mlab.figure(bgcolor=(0, 0, 0))

# Create a Mayavi contour3d visualization with 50% opacity
contour = mlab.contour3d(data, colormap="copper", opacity=0.5)

# Customize other visualization settings as needed

# Show the visualization
mlab.show()





In [2]:
## Creates Cube model using VTK Library

import vtkmodules.all as vtk

# Create a cube source
cube_source = vtk.vtkCubeSource()
cube_source.SetXLength(1.0)
cube_source.SetYLength(1.0)
cube_source.SetZLength(1.0)
# Create a mapper
cube_mapper = vtk.vtkPolyDataMapper()
cube_mapper.SetInputConnection(cube_source.GetOutputPort())
# Create an actor
cube_actor = vtk.vtkActor()
cube_actor.SetMapper(cube_mapper)
# Create a renderer
renderer = vtk.vtkRenderer()
# Create a render window
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
# Create a render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)
# Add the cube actor to the renderer
renderer.AddActor(cube_actor)
# Set the background color
renderer.SetBackground(0.1, 0.1, 0.1)
# Reset the camera and render
renderer.ResetCamera()
render_window.Render()
# Save the cube as a VTK file
vtk_writer = vtk.vtkPolyDataWriter()
vtk_writer.SetFileName("cube.vtk")
vtk_writer.SetInputData(cube_source.GetOutput())
vtk_writer.Write()
